## Install minari

### Subtask:
Install the `minari` library and its dependencies.


# import libs


**Reasoning**:
The subtask is to install the `gymnasium` library and its dependencies. We will also be using `pytorch` for writing the transformer and training code



In [ ]:
import os
import sys
import random
import csv
from datetime import datetime
import pickle
import collections
import math

import numpy as np
import gymnasium as gym

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
# import gymnasium as gym
# import minari
# !minari download mujoco/hopper/expert-v0
# dataset = minari.load_dataset('mujoco/hopper/expert-v0')
# print("Observation space:", dataset.observation_space)
# print("Action space:", dataset.action_space)
# print("Total episodes:", dataset.total_episodes)
# print("Total steps:", dataset.total_steps)
# episodes = list(minari.load_dataset('mujoco/hopper/expert-v0').iterate_episodes())

# # Iterate through the first few episodes and print their attributes
# print("Inspecting attributes of the first 5 episodes:")
# for i, episode in enumerate(list(dataset.iterate_episodes())[:5]):
#     print(f"\nEpisode {i+1}:")
#     print("Available attributes:")
#     for attr_name in dir(episode):
#         if not attr_name.startswith('_'):
#             try:
#                 attr_value = getattr(episode, attr_name)
#                 print(f"  - {attr_name}: {type(attr_value)}")
#             except AttributeError:
#                 print(f"  - {attr_name}: [Error accessing attribute]")

namespace_metadata.json: 1.31kB [00:00, 3.25MB/s]
metadata.json: 1.59kB [00:00, 9.76MB/s]
namespace_metadata.json: 1.68kB [00:00, 7.83MB/s]
namespace_metadata.json: 10.9kB [00:00, 37.7MB/s]
namespace_metadata.json: 3.26kB [00:00, 14.1MB/s]
namespace_metadata.json: 3.62kB [00:00, 9.66MB/s]
namespace_metadata.json: 100% 238/238 [00:00<00:00, 1.98MB/s]
namespace_metadata.json: 1.31kB [00:00, 6.10MB/s]
namespace_metadata.json: 100% 110/110 [00:00<00:00, 834kB/s]

Fetching 2 files:   0% 0/2 [00:00<?, ?it/s]
main_data.hdf5:   0% 0.00/135M [00:00<?, ?B/s]
main_data.hdf5:  16% 21.0M/135M [00:00<00:00, 163MB/s]
main_data.hdf5:  31% 41.9M/135M [00:00<00:00, 182MB/s]
main_data.hdf5:  47% 62.9M/135M [00:00<00:00, 180MB/s]
main_data.hdf5:  62% 83.9M/135M [00:00<00:00, 178MB/s]
main_data.hdf5:  78% 105M/135M [00:00<00:00, 188MB/s] 
main_data.hdf5: 100% 135M/135M [00:00<00:00, 185MB/s]
Fetching 2 files: 100% 2/2 [00:00<00:00,  2.25it/s]

Dataset mujoco/hopper/expert-v0 downloaded to /root/.minari/dat

# training parameters

In [ ]:
# import minari
# dataset = "expert"       # expert / medium / random (adjust based on available Minari datasets)
# rtg_scale = 1000                # scale to normalize returns to go

# # use v3 env for evaluation because
# # DT paper evaluates results on v3 envs

# env_name = 'Hopper-v5'
# rtg_target = 4500
# env_d4rl_name = 'mujoco/hopper/expert-v0'  # Now using Minari format

# # env_name = 'HalfCheetah-v3'
# # rtg_target = 6000
# # env_d4rl_name = 'mujoco/halfcheetah/expert-v0'

# # env_name = 'Walker2d-v3'
# # rtg_target = 5000
# # env_d4rl_name = 'mujoco/walker2d/expert-v0'

# max_eval_ep_len = 1000      # max len of one evaluation episode
# num_eval_ep = 10            # num of evaluation episodes per iteration

# batch_size = 64             # training batch size
# lr = 1e-4                   # learning rate
# wt_decay = 1e-4             # weight decay
# warmup_steps = 7500        # warmup steps for lr scheduler

# # total updates = max_train_iters x num_updates_per_iter
# max_train_iters = 500
# num_updates_per_iter = 100

# context_len = 20        # K in decision transformer
# n_blocks = 3            # num of transformer blocks
# embed_dim = 128         # embedding (hidden) dim of transformer
# n_heads = 1             # num of transformer heads
# dropout_p = 0.1         # dropout probability

# # load data from this file
# dataset_path = env_d4rl_name  # Now points to Minari dataset name instead of file path

# # saves model and csv in this directory
# log_dir = "./dt_runs/"

# if not os.path.exists(log_dir):
#     os.makedirs(log_dir)

# # training and evaluation device
# device_name = 'cuda'
# device = torch.device(device_name)
# print("device set to: ", device)

device set to:  cuda


### Identical hyperparameters used for the lunar lander env

In [ ]:
# --- Training Parameters for LunarLander-v2 ---

# Data generation
num_episodes_data = 500       # Number of episodes to generate for the offline dataset

# Environment settings
env_name = 'LunarLander-v3'
rtg_target = 100
# Scale rewards to a reasonable range for the model
rtg_scale = 100

# Evaluation settings
max_eval_ep_len = 1000      # Max length of one evaluation episode
num_eval_ep = 10            # Num of evaluation episodes per iteration

# Training settings
batch_size = 64             # Training batch size
lr = 1e-4                   # Learning rate
wt_decay = 1e-4             # Weight decay
warmup_steps = 7500         # Warmup steps for lr scheduler

# Total updates = max_train_iters x num_updates_per_iter
max_train_iters = 500
num_updates_per_iter = 100

# Model hyperparameters
context_len = 20        # K in decision transformer
n_blocks = 3            # Num of transformer blocks
embed_dim = 128         # Embedding (hidden) dim of transformer
n_heads = 1             # Num of transformer heads
dropout_p = 0.1         # Dropout probability

# Logging
log_dir = "./dt_runs_lunarlander/"

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device set to: {device}")

# decision transformer model

In [ ]:

"""
this extremely minimal GPT model is based on:
Misha Laskin's tweet:
https://twitter.com/MishaLaskin/status/1481767788775628801?cxt=HHwWgoCzmYD9pZApAAAA

and its corresponding notebook:
https://colab.research.google.com/drive/1NUBqyboDcGte5qAJKOl8gaJC28V_73Iv?usp=sharing

the above colab has a bug while applying masked_fill which is fixed in the
following code

"""

class MaskedCausalAttention(nn.Module):
    def __init__(self, h_dim, max_T, n_heads, drop_p):
        super().__init__()
        self.n_heads = n_heads
        self.max_T = max_T
        self.q_net = nn.Linear(h_dim, h_dim)
        self.k_net = nn.Linear(h_dim, h_dim)
        self.v_net = nn.Linear(h_dim, h_dim)
        self.proj_net = nn.Linear(h_dim, h_dim)
        self.att_drop = nn.Dropout(drop_p)
        self.proj_drop = nn.Dropout(drop_p)
        ones = torch.ones((max_T, max_T))
        mask = torch.tril(ones).view(1, 1, max_T, max_T)
        # register buffer makes sure mask does not get updated
        # during backpropagation
        self.register_buffer('mask',mask)

    def forward(self, x):
        B, T, C = x.shape # batch size, seq length, h_dim * n_heads
        N, D = self.n_heads, C // self.n_heads # N = num heads, D = attention dim
        # rearrange q, k, v as (B, N, T, D)
        q = self.q_net(x).view(B, T, N, D).transpose(1,2)
        k = self.k_net(x).view(B, T, N, D).transpose(1,2)
        v = self.v_net(x).view(B, T, N, D).transpose(1,2)
        # weights (B, N, T, T)
        weights = q @ k.transpose(2,3) / math.sqrt(D)
        # causal mask applied to weights
        weights = weights.masked_fill(self.mask[...,:T,:T] == 0, float('-inf'))
        # normalize weights, all -inf -> 0 after softmax
        normalized_weights = F.softmax(weights, dim=-1)
        # attention (B, N, T, D)
        attention = self.att_drop(normalized_weights @ v)
        # gather heads and project (B, N, T, D) -> (B, T, N*D)
        attention = attention.transpose(1, 2).contiguous().view(B,T,N*D)
        out = self.proj_drop(self.proj_net(attention))
        return out


class Block(nn.Module):
    def __init__(self, h_dim, max_T, n_heads, drop_p):
        super().__init__()
        self.attention = MaskedCausalAttention(h_dim, max_T, n_heads, drop_p)
        self.mlp = nn.Sequential(
                nn.Linear(h_dim, 4*h_dim),
                nn.GELU(),
                nn.Linear(4*h_dim, h_dim),
                nn.Dropout(drop_p),
            )
        self.ln1 = nn.LayerNorm(h_dim)
        self.ln2 = nn.LayerNorm(h_dim)

    def forward(self, x):
        # Attention -> LayerNorm -> MLP -> LayerNorm
        x = x + self.attention(x) # residual
        x = self.ln1(x)
        x = x + self.mlp(x) # residual
        x = self.ln2(x)
        return x


class DecisionTransformer(nn.Module):
    def __init__(self, state_dim, act_dim, n_blocks, h_dim, context_len,
                 n_heads, drop_p, max_timestep=4096):
        super().__init__()

        self.state_dim = state_dim
        self.act_dim = act_dim
        self.h_dim = h_dim
        ### transformer blocks
        input_seq_len = 3 * context_len
        blocks = [Block(h_dim, input_seq_len, n_heads, drop_p) for _ in range(n_blocks)]
        self.transformer = nn.Sequential(*blocks)
        ### projection heads (project to embedding)
        self.embed_ln = nn.LayerNorm(h_dim)
        self.embed_timestep = nn.Embedding(max_timestep, h_dim)
        self.embed_rtg = torch.nn.Linear(1, h_dim)
        self.embed_state = torch.nn.Linear(state_dim, h_dim)
        # # discrete actions
        # self.embed_action = torch.nn.Embedding(act_dim, h_dim)
        # use_action_tanh = False # False for discrete actions
        # continuous actions
        self.embed_action = torch.nn.Linear(act_dim, h_dim)
        use_action_tanh = True # True for continuous actions
        ### prediction heads
        self.predict_rtg = torch.nn.Linear(h_dim, 1)
        self.predict_state = torch.nn.Linear(h_dim, state_dim)
        self.predict_action = nn.Sequential(
            *([nn.Linear(h_dim, act_dim)] + ([nn.Tanh()] if use_action_tanh else []))
        )


    def forward(self, timesteps, states, actions, returns_to_go):

        B, T, C = states.shape

        time_embeddings = self.embed_timestep(timesteps)

        # time embeddings are treated similar to positional embeddings
        state_embeddings = self.embed_state(states) + time_embeddings
        action_embeddings = self.embed_action(actions) + time_embeddings
        returns_embeddings = self.embed_rtg(returns_to_go) + time_embeddings

        # stack rtg, states and actions and reshape sequence as
        # (r1, s1, a1, r2, s2, a2 ...)
        h = torch.stack(
            (returns_embeddings, state_embeddings, action_embeddings), dim=1
        ).permute(0, 2, 1, 3).reshape(B, 3 * T, self.h_dim)

        h = self.embed_ln(h)

        # transformer and prediction
        h = self.transformer(h)

        # get h reshaped such that its size = (B x 3 x T x h_dim) and
        # h[:, 0, t] is conditioned on r_0, s_0, a_0 ... r_t
        # h[:, 1, t] is conditioned on r_0, s_0, a_0 ... r_t, s_t
        # h[:, 2, t] is conditioned on r_0, s_0, a_0 ... r_t, s_t, a_t
        h = h.reshape(B, T, 3, self.h_dim).permute(0, 2, 1, 3)

        # get predictions
        return_preds = self.predict_rtg(h[:,2])     # predict next rtg given r, s, a
        state_preds = self.predict_state(h[:,2])    # predict next state given r, s, a
        action_preds = self.predict_action(h[:,1])  # predict action given r, s

        return state_preds, action_preds, return_preds

In [ ]:
class MaskedCausalAttention(nn.Module):
    def __init__(self, h_dim, max_T, n_heads, drop_p):
        super().__init__()
        self.n_heads = n_heads
        self.max_T = max_T
        self.q_net = nn.Linear(h_dim, h_dim)
        self.k_net = nn.Linear(h_dim, h_dim)
        self.v_net = nn.Linear(h_dim, h_dim)
        self.proj_net = nn.Linear(h_dim, h_dim)
        self.att_drop = nn.Dropout(drop_p)
        self.proj_drop = nn.Dropout(drop_p)
        ones = torch.ones((max_T, max_T))
        mask = torch.tril(ones).view(1, 1, max_T, max_T)
        self.register_buffer('mask', mask)

    def forward(self, x):
        B, T, C = x.shape
        N, D = self.n_heads, C // self.n_heads
        q = self.q_net(x).view(B, T, N, D).transpose(1, 2)
        k = self.k_net(x).view(B, T, N, D).transpose(1, 2)
        v = self.v_net(x).view(B, T, N, D).transpose(1, 2)
        weights = q @ k.transpose(2, 3) / math.sqrt(D)
        weights = weights.masked_fill(self.mask[..., :T, :T] == 0, float('-inf'))
        normalized_weights = F.softmax(weights, dim=-1)
        attention = self.att_drop(normalized_weights @ v)
        attention = attention.transpose(1, 2).contiguous().view(B, T, N * D)
        out = self.proj_drop(self.proj_net(attention))
        return out

class Block(nn.Module):
    def __init__(self, h_dim, max_T, n_heads, drop_p):
        super().__init__()
        self.attention = MaskedCausalAttention(h_dim, max_T, n_heads, drop_p)
        self.mlp = nn.Sequential(
            nn.Linear(h_dim, 4 * h_dim),
            nn.GELU(),
            nn.Linear(4 * h_dim, h_dim),
            nn.Dropout(drop_p),
        )
        self.ln1 = nn.LayerNorm(h_dim)
        self.ln2 = nn.LayerNorm(h_dim)

    def forward(self, x):
        x = x + self.attention(x)
        x = self.ln1(x)
        x = x + self.mlp(x)
        x = self.ln2(x)
        return x

class DecisionTransformer(nn.Module):
    def __init__(self, state_dim, act_dim, n_blocks, h_dim, context_len,
                 n_heads, drop_p, max_timestep=4096, is_discrete=True):
        super().__init__()
        self.state_dim = state_dim
        self.act_dim = act_dim
        self.h_dim = h_dim
        self.is_discrete = is_discrete

        input_seq_len = 3 * context_len
        blocks = [Block(h_dim, input_seq_len, n_heads, drop_p) for _ in range(n_blocks)]
        self.transformer = nn.Sequential(*blocks)

        self.embed_ln = nn.LayerNorm(h_dim)
        self.embed_timestep = nn.Embedding(max_timestep, h_dim)
        self.embed_rtg = torch.nn.Linear(1, h_dim)
        self.embed_state = torch.nn.Linear(state_dim, h_dim)

        if self.is_discrete:
            self.embed_action = torch.nn.Embedding(act_dim, h_dim)
        else:
            self.embed_action = torch.nn.Linear(act_dim, h_dim)

        self.predict_rtg = torch.nn.Linear(h_dim, 1)
        self.predict_state = torch.nn.Linear(h_dim, state_dim)
        if self.is_discrete:
            self.predict_action = nn.Linear(h_dim, act_dim)
        else:
            self.predict_action = nn.Sequential(
                nn.Linear(h_dim, act_dim),
                nn.Tanh()
            )

    def forward(self, timesteps, states, actions, returns_to_go):
        B, T, _ = states.shape
        time_embeddings = self.embed_timestep(timesteps)
        state_embeddings = self.embed_state(states) + time_embeddings
        returns_embeddings = self.embed_rtg(returns_to_go) + time_embeddings

        if self.is_discrete:
            action_embeddings = self.embed_action(actions.squeeze(-1)) + time_embeddings
        else:
            action_embeddings = self.embed_action(actions) + time_embeddings

        h = torch.stack(
            (returns_embeddings, state_embeddings, action_embeddings), dim=1
        ).permute(0, 2, 1, 3).reshape(B, 3 * T, self.h_dim)
        h = self.embed_ln(h)
        h = self.transformer(h)
        h = h.reshape(B, T, 3, self.h_dim).permute(0, 2, 1, 3)

        return_preds = self.predict_rtg(h[:, 2])
        state_preds = self.predict_state(h[:, 2])
        action_preds = self.predict_action(h[:, 1])

        return state_preds, action_preds, return_preds

# infos

In [ ]:
REF_MAX_SCORE = {}
REF_MIN_SCORE = {}
DATASET_STATS = {}

def compute_dataset_stats(env_d4rl_name):
    """Compute statistics from Minari dataset and process trajectories in one pass"""
    dataset = minari.load_dataset(env_d4rl_name)

    all_states = []
    all_rewards = []
    episode_returns = []
    trajectories = []

    print(f"Computing statistics for {env_d4rl_name}...")
    print(f"Total episodes: {dataset.total_episodes}")
    print(f"Total steps: {dataset.total_steps}")

    for episode in dataset.iterate_episodes():
        # Handle episode data
        obs = episode.observations.astype(np.float32)
        acts = episode.actions.astype(np.float32)
        rews = episode.rewards.astype(np.float32)

        # Handle length mismatches
        min_len = min(len(obs), len(acts), len(rews))

        # Collect RAW states for normalization statistics
        all_states.append(obs[:min_len])

        # Collect rewards for min/max computation
        all_rewards.extend(rews[:min_len])

        # Compute episode return
        episode_return = np.sum(rews[:min_len])
        episode_returns.append(episode_return)

        # Create trajectory (keep RAW observations for now)
        traj = {
            'observations': obs[:min_len],
            'actions': acts[:min_len],
            'rewards': rews[:min_len],
            'terminals': episode.terminations[:min_len].astype(np.bool_),
            'timeouts': episode.truncations[:min_len].astype(np.bool_)
        }

        trajectories.append(traj)

    # Compute state statistics on RAW data
    all_states = np.concatenate(all_states, axis=0).astype(np.float32)
    state_mean = np.mean(all_states, axis=0)
    state_std = np.std(all_states, axis=0) + 1e-6

    # Compute score statistics
    episode_returns = np.array(episode_returns)
    max_score = np.max(episode_returns)
    min_score = np.min(episode_returns)

    # Get environment key for compatibility
    env_key = env_d4rl_name.split('/')[1]  # e.g., 'hopper' from 'mujoco/hopper/expert-v0'

    # Update global dictionaries
    REF_MAX_SCORE[env_key] = max_score
    REF_MIN_SCORE[env_key] = min_score
    DATASET_STATS[env_d4rl_name] = {
        'state_mean': state_mean.tolist(),
        'state_std': state_std.tolist()
    }

    print(f"Environment: {env_key}")
    print(f"Max score: {max_score}")
    print(f"Min score: {min_score}")
    print(f"Average return: {np.mean(episode_returns):.2f}")
    print(f"State dim: {len(state_mean)}")

    return state_mean, state_std, min_score, max_score, trajectories

# utils

In [ ]:
from torch.utils.data import Dataset

def discount_cumsum(x, gamma):
    disc_cumsum = np.zeros_like(x)
    disc_cumsum[-1] = x[-1]
    for t in reversed(range(x.shape[0]-1)):
        disc_cumsum[t] = x[t] + gamma * disc_cumsum[t+1]
    return disc_cumsum

def get_d4rl_dataset_stats(env_d4rl_name):
    return DATASET_STATS[env_d4rl_name]

def get_d4rl_normalized_score(score, env_name):
    env_key = env_name.split('-')[0].lower()
    assert env_key in REF_MAX_SCORE, f'no reference score for {env_key} env to calculate d4rl score'
    return (score - REF_MIN_SCORE[env_key]) / (REF_MAX_SCORE[env_key] - REF_MIN_SCORE[env_key])

def evaluate_on_env(model, device, context_len, env, rtg_target, rtg_scale,
                    num_eval_ep=10, max_test_ep_len=1000,
                    state_mean=None, state_std=None, render=False):

    eval_batch_size = 1  # required for forward pass

    results = {}
    total_reward = 0
    total_timesteps = 0

    state_dim = env.observation_space.shape[0]
    act_dim = env.action_space.shape[0]

    if state_mean is None:
        state_mean = torch.zeros((state_dim,)).to(device)
    else:
        state_mean = torch.from_numpy(state_mean).to(device)

    if state_std is None:
        state_std = torch.ones((state_dim,)).to(device)
    else:
        state_std = torch.from_numpy(state_std).to(device)

    # same as timesteps used for training the transformer
    # also, crashes if device is passed to arange()
    timesteps = torch.arange(start=0, end=max_test_ep_len, step=1)
    timesteps = timesteps.repeat(eval_batch_size, 1).to(device)

    model.eval()

    with torch.no_grad():

        for _ in range(num_eval_ep):

            # zeros place holders
            actions = torch.zeros((eval_batch_size, max_test_ep_len, act_dim),
                                dtype=torch.float32, device=device)

            states = torch.zeros((eval_batch_size, max_test_ep_len, state_dim),
                                dtype=torch.float32, device=device)

            rewards_to_go = torch.zeros((eval_batch_size, max_test_ep_len, 1),
                                dtype=torch.float32, device=device)

            # init episode
            reset_result = env.reset()
            if isinstance(reset_result, tuple):
                running_state = reset_result[0]  # Gymnasium returns (obs, info)
            else:
                running_state = reset_result  # Old gym returns just obs
            running_reward = 0
            running_rtg = rtg_target / rtg_scale

            for t in range(max_test_ep_len):

                total_timesteps += 1

                # add state in placeholder and normalize
                states[0, t] = torch.from_numpy(running_state).to(device)
                states[0, t] = (states[0, t] - state_mean) / state_std

                # calcualate running rtg and add in placeholder
                running_rtg = running_rtg - (running_reward / rtg_scale)
                rewards_to_go[0, t] = running_rtg

                if t < context_len:
                    _, act_preds, _ = model.forward(timesteps[:,:context_len],
                                                states[:,:context_len],
                                                actions[:,:context_len],
                                                rewards_to_go[:,:context_len])
                    act = act_preds[0, t].detach()
                else:
                    _, act_preds, _ = model.forward(timesteps[:,t-context_len+1:t+1],
                                                states[:,t-context_len+1:t+1],
                                                actions[:,t-context_len+1:t+1],
                                                rewards_to_go[:,t-context_len+1:t+1])
                    act = act_preds[0, -1].detach()

                step_result = env.step(act.cpu().numpy())
                if len(step_result) == 5:  # Gymnasium returns (obs, reward, terminated, truncated, info)
                    running_state, running_reward, terminated, truncated, _ = step_result
                    done = terminated or truncated
                elif len(step_result) == 4:  # Old gym returns (obs, reward, done, info)
                    running_state, running_reward, done, _ = step_result
                else:
                    raise ValueError(f"Unexpected step result length: {len(step_result)}")

                # add action in placeholder
                actions[0, t] = act

                total_reward += running_reward

                if render:
                    env.render()
                if done:
                    break

    results['eval/avg_reward'] = total_reward / num_eval_ep
    results['eval/avg_ep_len'] = total_timesteps / num_eval_ep

    return results

# dataset

In [ ]:
state_mean, state_std, min_score, max_score, _ = compute_dataset_stats(env_d4rl_name)

print(f"\nDataset: {env_d4rl_name}")
print("State mean: ", state_mean.tolist())
print("State std: ", state_std.tolist())
print("Min score: ", min_score)
print("Max score: ", max_score)

In [ ]:
class D4RLTrajectoryDataset(Dataset):
    def __init__(self, dataset_path, context_len, rtg_scale):

        self.context_len = context_len

        # Load and process dataset in one pass (avoid double loading)
        print(f"Loading {dataset_path}...")
        state_mean_computed, state_std_computed, min_score, max_score, self.trajectories = compute_dataset_stats(dataset_path)

        # Store computed statistics
        self.state_mean = state_mean_computed.astype(np.float32)
        self.state_std = state_std_computed.astype(np.float32)

        # Process trajectories: compute RTG and normalize states
        for traj in self.trajectories:
            # Calculate returns to go and rescale them
            returns_to_go = discount_cumsum(traj['rewards'], 1.0) / rtg_scale
            traj['returns_to_go'] = returns_to_go

            # Normalize states using computed statistics
            traj['observations'] = (traj['observations'] - self.state_mean) / self.state_std

        print(f"Loaded {len(self.trajectories)} trajectories")

        # Debug: Check shapes of first trajectory
        if len(self.trajectories) > 0:
            first_traj = self.trajectories[0]
            print(f"First trajectory shapes:")
            print(f"  observations: {first_traj['observations'].shape}")
            print(f"  actions: {first_traj['actions'].shape}")
            print(f"  returns_to_go: {first_traj['returns_to_go'].shape}")

    def get_state_stats(self):
        return self.state_mean, self.state_std

    def __len__(self):
        return len(self.trajectories)

    def __getitem__(self, idx):
        traj = self.trajectories[idx]
        traj_len = traj['observations'].shape[0]

        if traj_len >= self.context_len:
            # sample random index to slice trajectory
            si = random.randint(0, traj_len - self.context_len)

            states = torch.from_numpy(traj['observations'][si : si + self.context_len])
            actions = torch.from_numpy(traj['actions'][si : si + self.context_len])
            returns_to_go = torch.from_numpy(traj['returns_to_go'][si : si + self.context_len])
            timesteps = torch.arange(start=si, end=si+self.context_len, step=1)

            # all ones since no padding
            traj_mask = torch.ones(self.context_len, dtype=torch.long)

        else:
            padding_len = self.context_len - traj_len

            # padding with zeros
            states = torch.from_numpy(traj['observations'])
            states = torch.cat([states,
                                torch.zeros(([padding_len] + list(states.shape[1:])),
                                dtype=states.dtype)],
                               dim=0)

            actions = torch.from_numpy(traj['actions'])
            actions = torch.cat([actions,
                                torch.zeros(([padding_len] + list(actions.shape[1:])),
                                dtype=actions.dtype)],
                               dim=0)

            returns_to_go = torch.from_numpy(traj['returns_to_go'])
            returns_to_go = torch.cat([returns_to_go,
                                torch.zeros(([padding_len] + list(returns_to_go.shape[1:])),
                                dtype=returns_to_go.dtype)],
                               dim=0)

            timesteps = torch.arange(start=0, end=self.context_len, step=1)

            traj_mask = torch.cat([torch.ones(traj_len, dtype=torch.long),
                                   torch.zeros(padding_len, dtype=torch.long)],
                                  dim=0)

        return timesteps, returns_to_go, actions, states, traj_mask

# train

In [ ]:
start_time = datetime.now().replace(microsecond=0)

start_time_str = start_time.strftime("%y-%m-%d-%H-%M-%S")

prefix = "dt_" + env_d4rl_name

save_model_name =  prefix + "_model_" + start_time_str + ".pt"
save_model_path = os.path.join(log_dir, save_model_name)
save_best_model_path = save_model_path[:-3] + "_best.pt"

log_csv_name = prefix + "_log_" + start_time_str + ".csv"
log_csv_path = os.path.join(log_dir, log_csv_name)

os.makedirs(os.path.dirname(log_csv_path), exist_ok=True)
csv_writer = csv.writer(open(log_csv_path, 'a', 1))
csv_header = (["duration", "num_updates", "action_loss",
               "eval_avg_reward", "eval_avg_ep_len", "eval_d4rl_score"])

csv_writer.writerow(csv_header)
os.makedirs(os.path.dirname(save_model_path), exist_ok=True)


print("=" * 60)
print("start time: " + start_time_str)
print("=" * 60)

print("device set to: " + str(device))
print("dataset path: " + dataset_path)
print("model save path: " + save_model_path)
# print("log csv save path: " + log_csv_path)

# Compute dataset statistics before creating dataset
print("Computing dataset statistics...")
state_mean_computed, state_std_computed, min_score, max_score, _ = compute_dataset_stats(dataset_path)

traj_dataset = D4RLTrajectoryDataset(dataset_path, context_len, rtg_scale)

traj_data_loader = DataLoader(traj_dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        pin_memory=True,
                        drop_last=True)

data_iter = iter(traj_data_loader)

## get state stats from dataset
state_mean, state_std = traj_dataset.get_state_stats()

# Create environment - use gymnasium instead of gym for compatibility
try:
    import gymnasium as gym
    env = gym.make(env_name)
except ImportError:
    import gym
    env = gym.make(env_name)

state_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

model = DecisionTransformer(
            state_dim=state_dim,
            act_dim=act_dim,
            n_blocks=n_blocks,
            h_dim=embed_dim,
            context_len=context_len,
            n_heads=n_heads,
            drop_p=dropout_p,
        ).to(device)

optimizer = torch.optim.AdamW(
                    model.parameters(),
                    lr=lr,
                    weight_decay=wt_decay
                )

scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer,
        lambda steps: min((steps+1)/warmup_steps, 1)
    )

max_d4rl_score = -1.0
total_updates = 0

for i_train_iter in range(max_train_iters):

    log_action_losses = []
    model.train()

    for _ in range(num_updates_per_iter):
        try:
            timesteps, states, actions, returns_to_go, traj_mask = next(data_iter)
        except StopIteration:
            data_iter = iter(traj_data_loader)
            timesteps, states, actions, returns_to_go, traj_mask = next(data_iter)

        timesteps = timesteps.to(device)    # B x T
        states = states.to(device)          # B x T x state_dim
        actions = actions.to(device)        # B x T x act_dim
        returns_to_go = returns_to_go.to(device).unsqueeze(dim=-1) # B x T x 1
        traj_mask = traj_mask.to(device)    # B x T

        action_target = torch.clone(actions).detach().to(device)

        state_preds, action_preds, return_preds = model.forward(
                                                        timesteps=timesteps,
                                                        states=states,
                                                        actions=actions,
                                                        returns_to_go=returns_to_go
                                                    )

        # only consider non padded elements
        action_preds = action_preds.view(-1, act_dim)[traj_mask.view(-1,) > 0]
        action_target = action_target.view(-1, act_dim)[traj_mask.view(-1,) > 0]

        action_loss = F.mse_loss(action_preds, action_target, reduction='mean')

        optimizer.zero_grad()
        action_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        optimizer.step()
        scheduler.step()

        log_action_losses.append(action_loss.detach().cpu().item())

    # evaluate on env
    results = evaluate_on_env(model, device, context_len, env, rtg_target, rtg_scale,
                            num_eval_ep, max_eval_ep_len, state_mean, state_std,
                            )
    eval_avg_reward = results['eval/avg_reward']
    eval_avg_ep_len = results['eval/avg_ep_len']
    eval_d4rl_score = get_d4rl_normalized_score(results['eval/avg_reward'], env_name) * 100

    mean_action_loss = np.mean(log_action_losses)
    time_elapsed = str(datetime.now().replace(microsecond=0) - start_time)

    total_updates += num_updates_per_iter

    log_str = ("=" * 60 + '\n' +
            "time elapsed: " + time_elapsed  + '\n' +
            "num of updates: " + str(total_updates) + '\n' +
            "action loss: " +  format(mean_action_loss, ".5f") + '\n' +
            "eval avg reward: " + format(eval_avg_reward, ".5f") + '\n' +
            "eval avg ep len: " + format(eval_avg_ep_len, ".5f") + '\n' +
            "eval d4rl score: " + format(eval_d4rl_score, ".5f")
            )

    print(log_str)

    log_data = [time_elapsed, total_updates, mean_action_loss,
                eval_avg_reward, eval_avg_ep_len,
                eval_d4rl_score]

    # csv_writer.writerow(log_data)

    # save model
    print("max d4rl score: " + format(max_d4rl_score, ".5f"))
    if eval_d4rl_score >= max_d4rl_score:
        print("saving max d4rl score model at: " + save_best_model_path)
        torch.save(model.state_dict(), save_best_model_path)
        max_d4rl_score = eval_d4rl_score

    print("saving current model at: " + save_model_path)
    torch.save(model.state_dict(), save_model_path)


print("=" * 60)
print("finished training!")
print("=" * 60)
end_time = datetime.now().replace(microsecond=0)
time_elapsed = str(end_time - start_time)
end_time_str = end_time.strftime("%y-%m-%d-%H-%M-%S")
print("started training at: " + start_time_str)
print("finished training at: " + end_time_str)
print("total training time: " + time_elapsed)
print("max d4rl score: " + format(max_d4rl_score, ".5f"))
print("saved max d4rl score model at: " + save_best_model_path)
print("saved last updated model at: " + save_model_path)
print("=" * 60)

# csv_writer.close()

start time: 25-08-06-05-41-46
device set to: cuda
dataset path: mujoco/hopper/expert-v0
model save path: ./dt_runs/dt_mujoco/hopper/expert-v0_model_25-08-06-05-41-46.pt
Computing dataset statistics...
Computing statistics for mujoco/hopper/expert-v0...
Total episodes: 1086
Total steps: 999164
Environment: hopper
Max score: 4376.3271484375
Min score: 395.63897705078125
Average return: 3857.80
State dim: 11
Loading mujoco/hopper/expert-v0...
Computing statistics for mujoco/hopper/expert-v0...
Total episodes: 1086
Total steps: 999164
Environment: hopper
Max score: 4376.3271484375
Min score: 395.63897705078125
Average return: 3857.80
State dim: 11
Loaded 1086 trajectories
First trajectory shapes:
  observations: (1000, 11)
  actions: (1000, 3)
  returns_to_go: (1000,)


ValueError: not enough values to unpack (expected 3, got 2)

# test

In [ ]:
eval_dataset = "expert"  # expert / medium / random (adjust based on available Minari datasets)
eval_rtg_scale = 1000    # normalize returns to go
eval_env_name = "Hopper-v5"
eval_rtg_target = 5000
eval_env_d4rl_name = 'mujoco/hopper/expert-v0'  # Minari dataset name

# eval_env_name = "HalfCheetah-v3"
# eval_rtg_target = 6000
# eval_env_d4rl_name = 'mujoco/halfcheetah/expert-v0'

# eval_env_name = "Hopper-v3"
# eval_rtg_target = 3600
# eval_env_d4rl_name = 'mujoco/hopper/expert-v0'

num_test_eval_ep = 10        # num of evaluation episodes
eval_max_eval_ep_len = 1000  # max len of one episode

context_len = 20
n_blocks = 3
embed_dim = 128
n_heads = 1
dropout_p = 0.1

eval_chk_pt_dir = "./dt_runs/"
eval_chk_pt_name = "dt_mujoco/hopper/expert-v0_model_25-07-26-22-20-35_best.pt"  # Update with your actual checkpoint name
eval_chk_pt_list = [eval_chk_pt_name]

## manually override check point list
## passing a list will evaluate on all checkpoints
## and output mean and std score
# eval_chk_pt_list = [
#     "dt_mujoco/hopper/expert-v0_model_25-07-26-18-59-07_best.pt",
#     "dt_mujoco/hopper/expert-v0_model_25-07-26-19-15-30_best.pt",
#     "dt_mujoco/hopper/expert-v0_model_25-07-26-20-45-12_best.pt"
# ]

# Compute dataset statistics if not already computed
# Check if the dataset stats exist, if not compute them
if eval_env_d4rl_name not in DATASET_STATS:
    print(f"Computing statistics for evaluation dataset: {eval_env_d4rl_name}")
    compute_dataset_stats(eval_env_d4rl_name)

env_data_stats = get_d4rl_dataset_stats(eval_env_d4rl_name)
eval_state_mean = np.array(env_data_stats['state_mean'], dtype=np.float32)
eval_state_std = np.array(env_data_stats['state_std'], dtype=np.float32)

try:
    import gymnasium as gym
    eval_env = gym.make(eval_env_name)
except ImportError:
    import gym
    eval_env = gym.make(eval_env_name)

state_dim = eval_env.observation_space.shape[0]
act_dim = eval_env.action_space.shape[0]

all_scores = []

for eval_chk_pt_name in eval_chk_pt_list:
    eval_model = DecisionTransformer(
        state_dim=state_dim,
        act_dim=act_dim,
        n_blocks=n_blocks,
        h_dim=embed_dim,
        context_len=context_len,
        n_heads=n_heads,
        drop_p=dropout_p,
    ).to(device)

    eval_chk_pt_path = os.path.join(eval_chk_pt_dir, eval_chk_pt_name)

    eval_model.load_state_dict(torch.load(eval_chk_pt_path, map_location=device))
    print("model loaded from: " + eval_chk_pt_path)

    results = evaluate_on_env(eval_model, device, context_len,
                             eval_env, eval_rtg_target, eval_rtg_scale,
                             num_test_eval_ep, eval_max_eval_ep_len,
                             eval_state_mean, eval_state_std)
    print(results)

    norm_score = get_d4rl_normalized_score(results['eval/avg_reward'], eval_env_name) * 100
    print("normalized d4rl score: ", norm_score)

    all_scores.append(norm_score)

print("=" * 60)
all_scores = np.array(all_scores)
print("evaluated on env: " + eval_env_name)
print("total num of checkpoints evaluated: " + str(len(eval_chk_pt_list)))
print("d4rl score mean: " + format(all_scores.mean(), ".5f"))
print("d4rl score std: " + format(all_scores.std(), ".5f"))
print("d4rl score var: " + format(all_scores.var(), ".5f"))
print("=" * 60)

## render env



*   saves mp4 video of env frames and plays it in notebook


In [ ]:


!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install -U colabgymrender


In [ ]:

from colabgymrender.recorder import Recorder

num_test_eval_ep = 1
eval_max_ep_len = 1000


directory = "./render_video"
eval_env = Recorder(eval_env, directory)

results = evaluate_on_env(eval_model, device, context_len,
                        eval_env, eval_rtg_target, eval_rtg_scale,
                        num_test_eval_ep, eval_max_ep_len,
						eval_state_mean, eval_state_std)
print(results)

norm_score = get_d4rl_normalized_score(results['eval/avg_reward'], eval_env_name) * 100
print("normalized d4rl score: ", norm_score)

eval_env.play()

eval_env.close()



# plots

In [ ]:
import numpy as np
import pickle

import pandas as pd
import glob

import matplotlib.pyplot as plt


env_d4rl_name = 'mujoco/hopper/expert-v0'

log_dir = 'dt_runs/'

x_key = "num_updates"
y_key = "eval_d4rl_score"
y_smoothing_win = 5
plot_avg = False
save_fig = False

if plot_avg:
    save_fig_path = env_d4rl_name + "_avg.png"
else:
    save_fig_path = env_d4rl_name + ".png"


all_files = glob.glob(log_dir + f'/dt_{env_d4rl_name}*.csv')

ax = plt.gca()
ax.set_title(env_d4rl_name)

if plot_avg:
    name_list = []
    df_list = []
    for filename in all_files:
        frame = pd.read_csv(filename, index_col=None, header=0)
        print(filename, frame.shape)
        frame['y_smooth'] = frame[y_key].rolling(window=y_smoothing_win).mean()
        df_list.append(frame)


    df_concat = pd.concat(df_list)
    df_concat_groupby = df_concat.groupby(df_concat.index)
    data_avg = df_concat_groupby.mean()

    data_avg.plot(x=x_key, y='y_smooth', ax=ax)

    ax.set_xlabel(x_key)
    ax.set_ylabel(y_key)
    ax.legend(['avg of all runs'], loc='lower right')

    if save_fig:
        plt.savefig(save_fig_path)

    plt.show()


else:
    name_list = []
    for filename in all_files:
        frame = pd.read_csv(filename, index_col=None, header=0)
        print(filename, frame.shape)
        frame['y_smooth'] = frame[y_key].rolling(window=y_smoothing_win).mean()
        frame.plot(x=x_key, y='y_smooth', ax=ax)
        name_list.append(filename.split('/')[-1])

    ax.set_xlabel(x_key)
    ax.set_ylabel(y_key)
    ax.legend(name_list, loc='lower right')

    if save_fig:
        plt.savefig(save_fig_path)

    plt.show()

